<a href="https://colab.research.google.com/github/dikoharyadhanto/AI-Deep-Learning-Documentation/blob/main/Klasifikasi_Gambar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Memastikan Versi TensorFlow 2.0 keatas**

In [5]:
import tensorflow as tf
print(tf.__version__)

2.6.0


## **Mempersiapkan Data**

In [6]:
!wget --no-check-certificate \
  https://github.com/dicodingacademy/assets/raw/main/ml_pemula_academy/messy-vs-clean-room.zip \
  -O /tmp/messy_vs_clean_room.zip

--2021-10-17 06:24:08--  https://github.com/dicodingacademy/assets/raw/main/ml_pemula_academy/messy-vs-clean-room.zip
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dicodingacademy/assets/main/ml_pemula_academy/messy-vs-clean-room.zip [following]
--2021-10-17 06:24:09--  https://raw.githubusercontent.com/dicodingacademy/assets/main/ml_pemula_academy/messy-vs-clean-room.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70392746 (67M) [application/zip]
Saving to: ‘/tmp/messy_vs_clean_room.zip’

/tmp/messy_vs_clean 100%[===================>]  67.13M   158MB/s    in 0.4s    

2021-10-17 06:24:14 

## **Mendefinisikan nama direktori untuk data latih dan data validasi.**

In [7]:
# melakukan ekstraksi pada file zip
import zipfile,os
local_zip = '/tmp/messy_vs_clean_room.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
 
base_dir = '/tmp/images'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')

## **Melihat struktur data latih dan data validasi**

In [8]:
os.listdir('/tmp/images/train')

['messy', 'clean']

In [9]:
os.listdir('/tmp/images/val')

['messy', 'clean']

## **Menampung direktori dari setiap kelas pada direktori latih dan direktori validasi ke dalam variabel.**

In [10]:
# membuat direktori ruangan rapi pada direktori data training
train_clean_dir = os.path.join(train_dir, 'clean')

# membuat direktori ruangan berantakan pada direktori data training
train_messy_dir = os.path.join(train_dir, 'messy')

# membuat direktori ruangan rapi pada direktori data validasi
val_clean_dir = os.path.join(validation_dir, 'clean')

# membuat direktori ruangan berantakan pada direktori data validasi
val_messy_dir = os.path.join(validation_dir, 'messy')

## **Augmentasi Gambar**

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')
 
test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

## **Mempersiapkan Data Latih Dengan Image Data Generator**

In [12]:
train_generator = train_datagen.flow_from_directory(
    train_dir,  # direktori data latih
    target_size=(150, 150),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
    batch_size=4,
    # karena kita merupakan masalah klasifikasi 2 kelas maka menggunakan class_mode = 'binary'
    class_mode='binary')
 
validation_generator = test_datagen.flow_from_directory(
    validation_dir, # direktori data validasi
    target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
    batch_size=4, 
    # karena kita merupakan masalah klasifikasi 2 kelas maka menggunakan class_mode = 'binary'
    class_mode='binary')

Found 192 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


## **Membangun Arsitektur CNN**

In [13]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

## **Memanggil fungsi compile pada objek model, dan tentukan loss function serta optimizer.**

In [14]:
# compile model dengan 'adam' optimizer loss function 'binary_crossentropy' 
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

## **Melatih model dengan metode fit**

In [15]:
# latih model dengan model.fit 
model.fit(
      train_generator,
      steps_per_epoch=25,  # berapa batch yang akan dieksekusi pada setiap epoch
      epochs=20, # tambahkan eposchs jika akurasi model belum optimal
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=5,  # berapa batch yang akan dieksekusi pada setiap epoch
      verbose=2)

Epoch 1/20
25/25 - 8s - loss: 0.7560 - accuracy: 0.4700 - val_loss: 0.6904 - val_accuracy: 0.5000
Epoch 2/20
25/25 - 6s - loss: 0.6949 - accuracy: 0.5100 - val_loss: 0.6928 - val_accuracy: 0.5000
Epoch 3/20
25/25 - 6s - loss: 0.6887 - accuracy: 0.6200 - val_loss: 0.6887 - val_accuracy: 0.5500
Epoch 4/20
25/25 - 6s - loss: 0.6828 - accuracy: 0.5300 - val_loss: 0.6856 - val_accuracy: 0.6500
Epoch 5/20
25/25 - 6s - loss: 0.6646 - accuracy: 0.6500 - val_loss: 0.6748 - val_accuracy: 0.5000
Epoch 6/20
25/25 - 6s - loss: 0.5983 - accuracy: 0.7300 - val_loss: 0.6314 - val_accuracy: 0.6500
Epoch 7/20
25/25 - 6s - loss: 0.6861 - accuracy: 0.5600 - val_loss: 0.6413 - val_accuracy: 0.6000
Epoch 8/20
25/25 - 6s - loss: 0.6668 - accuracy: 0.6200 - val_loss: 0.6095 - val_accuracy: 0.6500
Epoch 9/20
25/25 - 6s - loss: 0.6122 - accuracy: 0.6400 - val_loss: 0.6415 - val_accuracy: 0.6500
Epoch 10/20
25/25 - 6s - loss: 0.6071 - accuracy: 0.7300 - val_loss: 0.5820 - val_accuracy: 0.6500
Epoch 11/20
25/25 -

## **Melihat Hasil Model**

Kode di bawah memungkinkan kita untuk secara interaktif memilih sebuah berkas gambar, kemudian melakukan resize gambar dan mengubahnya menjadi larik numpy.

In [16]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
 
uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
 
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  
  print(fn)
  if classes==0:
    print('clean')
  else:
    print('messy')